### Goal
This code example shows you how to write custom evaluation functions to evaluate your models. Here we use an untrained ANN and evaluate on MNIST.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import devtorch

In [9]:
class ANNClassifier(devtorch.DevModel):
    
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.layer1 = nn.Linear(n_in, n_hidden, bias=False)
        self.layer2 = nn.Linear(n_hidden, n_out, bias=False)
        self.init_weight(self.layer1.weight, "glorot_uniform")
        self.init_weight(self.layer2.weight, "glorot_uniform")
    
    def forward(self, x):
        x = F.leaky_relu(self.layer1(x.flatten(1, 3)))
        return F.leaky_relu(self.layer2(x))

In [10]:
model = ANNClassifier(784, 2000, 10)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
test_dataset = datasets.MNIST("../../data", train=False, download=True, transform=transform)

### Eval error of untrained network

In [29]:
def error_metric(output, target):
    print(f"output={output.shape}")
    return (torch.max(output, 1)[1] == target).sum().cpu().item()

scores = devtorch.compute_metric(model, test_dataset, equal_to_three_metric, batch_size=256)
print(f"Accuracy = {1 - torch.Tensor(scores).sum()/len(test_dataset)}")

Accuracy = 0.8795999884605408


### Count number of predictions equal to 3

In [30]:
def equal_to_three_metric(output, target):
    return (torch.max(output, 1)[1] == 3).sum().cpu().item()
    
scores = devtorch.compute_metric(model, test_dataset, equal_to_three_metric, batch_size=256)
print(f"Equal to three = {torch.Tensor(scores).sum()}")

Equal to three = 1204.0
